# Role Based Access Control

Role based access control is a collection of privileges on resources given to roles. 

## Requirements
Before running the cells in this notebook, you must first execute all the cells in the following notebook:

* `01_Lab_Setup.ipynb`

## Getting Help
The best way to get help from the Yugabyte University team is to post your question on YugabyteDB Community Slack in the #training or #yb-university channels. To sign up, visit https://communityinviter.com/apps/yugabyte-db/register.

## Connect to YugabyteDB using the PostgreSQL Driver for Python
The following cells requires:
- Python 3.8+ and psycopg2

In [ ]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

# Role Based Access Control

Role based access control is a collection of privileges on resources given to roles. To see a list of roles currently available in your database, run the \du command.

In [ ]:
%sql \du

You will notice that there are some roles provided by default. These roles are used for various administrative purposes.

To create a new role, you can use the create role query. The following query creates a new role in the database named engineering.

In [ ]:
%sql create role engineering

The create role query will by default create a role that is not able to login to the database. If you want to allow the role to login, you can specify a method of authentication with the query. For example, you can create a role named John with a password authentication with the following query.

In [ ]:
%sql create role john login password 'yourpasswordhere'

In [ ]:
%sql grant engineering to john

In [ ]:
%sql \du

You can assign a role to another role. For example, if John is in the engineering department, you can assign him the engineering role
You can also revoke a role using a revoke query. For example, if John leaves the engineering department, you can remove the engineering role.

In [ ]:
%sql revoke engineering from john

In [ ]:
%sql \du

You can also delete a role from the database using a drop query.

In [ ]:
%sql drop role engineering

In [ ]:
%sql \du

## Granting Privileges

Once you have defined roles in your database, you are able to grant permissions to determine what each role can do in the database. To start, create a database and table.

In [ ]:
%sql create table test_table(id uuid,timeofdate timestamp,result boolean,details jsonb);

With the test table and database created, use the create role query to create some test roles in the database

In [ ]:
%sql create role engineering;
%sql create role developer;
%sql create role qa;
%sql create role db_admin;

If you want a role to have read access to test_table, you can use a grant select query. The following query grants select privileges on the test_table table to the engineering role.

In [ ]:
%sql grant select on test_table to engineering;

The \z command can be used to verify that the access privileges were set correctly. 

In [ ]:
%sql \z

There are various other grant queries which can be used to set privileges for roles. Insert, update, delete and truncate are examples of common grant queries. You can assign more than one privilege in a single query using a comma seperated list.

In [ ]:
%sql grant insert, update, delete, truncate on test_table to engineering;

In [ ]:
%sql \z

To allow a role to alter a table, you can assign the role owner using an alter table query.

In [ ]:
%sql alter table test_table owner to qa;

In [ ]:
%sql \z

To assign a role all privileges in a database, you can use alter role to assign the superuser privilege.  

In [ ]:
%sql alter role db_admin with superuser;

In [ ]:
%sql \z

After running the \du command, you will see the db_admin role assigned Superuser in the list of roles and attributes.

If you choose to remove the superuser role later, you can alter the role with the nosuperuser option.

In [ ]:
%sql alter role db_admin with nosuperuser;

## Check Access Script
Alternative tools like [CrunchyData](https://github.com/CrunchyData/crunchy_check_access) check access extension can be used to view access and permissions of various roles. This extension has been installed already in the notebook. To view all user permissions, run the query block below. 

In [ ]:
%sql SELECT * FROM all_access() WHERE base_role != CURRENT_USER;

This query displays all user access where the base role is not the current user, ignoring all system catalog and information schemas. This helps to filter the data displayed for permission queries, making it easier to find the data you need. Other functions like `my_privs` exist to allow a user to see their own permissions in the database. 

In [ ]:
%sql SELECT * FROM my_privs()

This query shows the permissions for our current user, which is `db_admin`. 

# All done!
In this lab, you completed the following:

* Roles
    * Created various roles in the database
    * Defined permissions for different roles in the database
    * Deleted and revoked roles

Next, run the following cell to open `03_RLS.ipynb`